# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,villa regina,-39.1000,-67.0667,297.58,23,74,3.13,AR,1737171234
1,1,bel ombre,-4.6167,55.4167,299.06,89,40,2.06,SC,1737171236
2,2,maryville,35.7565,-83.9705,279.87,68,100,0.00,US,1737171237
3,3,adamstown,-25.0660,-130.1015,297.95,88,6,4.71,PN,1737171238
4,4,wailua homesteads,22.0669,-159.3780,296.59,76,0,7.20,US,1737171198


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
plot_humid = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_humid

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=297)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=20)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AR')\
                            &(city_data_df['Country']!='SC')\
                            &(city_data_df['Country']!='PN')\
                            &(city_data_df['Humidity']<30),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head() 

6


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
211,211,new norfolk,-42.7826,147.0587,296.32,27,0,2.81,AU,1737171729
375,375,tongliao,43.6125,122.2653,275.09,28,10,4.18,CN,1737171765
424,424,deba,10.2086,11.3859,295.54,21,15,2.99,NG,1737171976
429,429,aoulef,26.9667,1.0833,282.85,29,19,4.04,DZ,1737171981
437,437,tamanrasset,22.7850,5.5228,284.10,12,0,3.09,DZ,1737171990


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head() 

,City,Country,Lat,Lng,Humidity,Hotel Name
211,new norfolk,AU,-42.7826,147.0587,27,
375,tongliao,CN,43.6125,122.2653,28,
424,deba,NG,10.2086,11.3859,21,
429,aoulef,DZ,26.9667,1.0833,29,
437,tamanrasset,DZ,22.7850,5.5228,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1} 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}' 

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
new norfolk - nearest hotel: Woodbridge on the Derwent
tongliao - nearest hotel: No hotel found
deba - nearest hotel: No hotel found
aoulef - nearest hotel: دار الضياف
tamanrasset - nearest hotel: فندق الأمان
windhoek - nearest hotel: Continental Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
211,new norfolk,AU,-42.7826,147.0587,27,Woodbridge on the Derwent
375,tongliao,CN,43.6125,122.2653,28,No hotel found
424,deba,NG,10.2086,11.3859,21,No hotel found
429,aoulef,DZ,26.9667,1.0833,29,دار الضياف
437,tamanrasset,DZ,22.7850,5.5228,12,فندق الأمان
583,windhoek,NaN,-22.5594,17.0832,27,Continental Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
plot_map = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)